# Assignment 4


#### (Significance threshold for entire assignment: .05)

In [1]:
import pylab as pl
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.formula.api as smf
# from getCitiBikeCSV import getCitiBikeCSV
%pylab inline
import os
import json
#s = json.load( open(os.getenv ('PUI2016')+"/fbb_matplotlibrc.json") )
#pl.rcParams.update(s)

Populating the interactive namespace from numpy and matplotlib


In [2]:
# importing one month from July and one from October
df = pd.read_csv("/gws/open/Student/citibike/201707-citibike-tripdata.csv.zip")
df1 = pd.read_csv("/gws/open/Student/citibike/201610-citibike-tripdata.zip")

In [3]:
# merging the two months into one dataset
df1.columns = df.columns
data = pd.concat([df, df1])
data.head(3)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,364,2017-07-01 00:00:00,2017-07-01 00:06:05,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,14744,Subscriber,1986.0,1
1,2142,2017-07-01 00:00:03,2017-07-01 00:35:46,293,Lafayette St & E 8 St,40.730207,-73.991026,3425,2 Ave & E 104 St,40.789210,-73.943708,19587,Subscriber,1981.0,1
2,328,2017-07-01 00:00:08,2017-07-01 00:05:37,3242,Schermerhorn St & Court St,40.691029,-73.991834,3397,Court St & Nelson St,40.676395,-73.998699,27937,Subscriber,1984.0,2


## Trip duration of bikers that ride during the day vs night. 
#### Null Hypothesis:
day = 7am - 7pm;
night = 7pm - 7am

Citi Bike duration is the same for riders during the day and during the night. 

$H_0$: D = N

In [4]:
# convert to starttime and add extract hour in new column
data['date'] = pd.to_datetime(data['starttime'])
data['hour'] = data['date'].dt.hour

In [5]:
def isDay(hour):
    if hour > 7 and hour < 19:
        return 1
    else:  
        return 0

In [6]:
# add column that adds day/night category
data['isDay'] = data['hour'].apply(isDay)
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,hour,isDay
0,364,2017-07-01 00:00:00,2017-07-01 00:06:05,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,14744,Subscriber,1986.0,1,2017-07-01 00:00:00,0,0
1,2142,2017-07-01 00:00:03,2017-07-01 00:35:46,293,Lafayette St & E 8 St,40.730207,-73.991026,3425,2 Ave & E 104 St,40.789210,-73.943708,19587,Subscriber,1981.0,1,2017-07-01 00:00:03,0,0
2,328,2017-07-01 00:00:08,2017-07-01 00:05:37,3242,Schermerhorn St & Court St,40.691029,-73.991834,3397,Court St & Nelson St,40.676395,-73.998699,27937,Subscriber,1984.0,2,2017-07-01 00:00:08,0,0
3,2530,2017-07-01 00:00:11,2017-07-01 00:42:22,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,26066,Subscriber,1985.0,1,2017-07-01 00:00:11,0,0
4,2534,2017-07-01 00:00:15,2017-07-01 00:42:29,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,29408,Subscriber,1982.0,2,2017-07-01 00:00:15,0,0


# KS Test

In [7]:
np.random.seed(14)

In [8]:
# testing if they're from the same distribution
day = data.tripduration[data['isDay'].values == 1]
night = data.tripduration[data['isDay'].values == 0]

#making a sample of day that's the same size as night - size(night)
Dsample = np.random.choice(day, size=918295, replace=False)

# 918295/200
Dsample200 = np.random.choice(Dsample, size=4591, replace=False)
Nsample200 = np.random.choice(night, size=4591, replace=False)


In [9]:
# test for all
scipy.stats.ks_2samp(day, night)

Ks_2sampResult(statistic=0.052998870130180231, pvalue=0.0)

In [10]:
# sample for 200
scipy.stats.ks_2samp(Dsample200, Nsample200)

Ks_2sampResult(statistic=0.066434328033108336, pvalue=2.8587064930076666e-09)

### What this means 
H0 - There is no statistical difference for trip duration for day and night riders. They come from the same distribution </br>

We reject the null, because the p value is zero. 

# Pearsons


In [11]:
# sort our samples, need to be the same size
Dsample_sorted = np.sort(Dsample)
Night_sorted = np.sort(night)

In [12]:
# same size samples
scipy.stats.pearsonr(Dsample_sorted, Night_sorted)

(0.87702608310598906, 0.0)

### What this means 
They're strongly correlated. We reject the null, because p is zero. <br/>


# Spearman's test

In [13]:
scipy.stats.spearmanr(Dsample_sorted, Night_sorted) 


SpearmanrResult(correlation=0.99999942813880904, pvalue=0.0)

### What this means
They're strongly correlated - We reject the null, because p is zero.

___


## Question: is there an age difference between brooklyn riders and manhattan riders? 

**Null:** The age of brooklyn riders and mahattan riders are the same. 


**Alternative:** The age of brooklyn riders is statistically different than manhattan riders


In [14]:
# interesting... there are probably NOT people over 100 riding citi bikes
data['age'] = 2017 - data['birth year']
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,hour,isDay,age
0,364,2017-07-01 00:00:00,2017-07-01 00:06:05,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,14744,Subscriber,1986.0,1,2017-07-01 00:00:00,0,0,31.0
1,2142,2017-07-01 00:00:03,2017-07-01 00:35:46,293,Lafayette St & E 8 St,40.730207,-73.991026,3425,2 Ave & E 104 St,40.789210,-73.943708,19587,Subscriber,1981.0,1,2017-07-01 00:00:03,0,0,36.0
2,328,2017-07-01 00:00:08,2017-07-01 00:05:37,3242,Schermerhorn St & Court St,40.691029,-73.991834,3397,Court St & Nelson St,40.676395,-73.998699,27937,Subscriber,1984.0,2,2017-07-01 00:00:08,0,0,33.0
3,2530,2017-07-01 00:00:11,2017-07-01 00:42:22,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,26066,Subscriber,1985.0,1,2017-07-01 00:00:11,0,0,32.0
4,2534,2017-07-01 00:00:15,2017-07-01 00:42:29,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,29408,Subscriber,1982.0,2,2017-07-01 00:00:15,0,0,35.0


In [15]:
# defining barriers for manhattan and brooklyn
# note: this will cut off some of the manhattan stations - hacky solution 
BatteryLat = 40.698413
WillyLong = -73.960669

In [16]:
def isBrooklyn (lat_long):
    latitude,longitude = lat_long
    if (latitude > BatteryLat) & (longitude > WillyLong):
        return 0
    else:
        return 1                   

In [17]:
# need double brackets here because passing in two things
data['isBrooklyn'] = data[["start station latitude", "start station longitude"]].apply(isBrooklyn, axis=1)    
data.head()     

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,hour,isDay,age,isBrooklyn
0,364,2017-07-01 00:00:00,2017-07-01 00:06:05,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,14744,Subscriber,1986.0,1,2017-07-01 00:00:00,0,0,31.0,0
1,2142,2017-07-01 00:00:03,2017-07-01 00:35:46,293,Lafayette St & E 8 St,40.730207,-73.991026,3425,2 Ave & E 104 St,40.789210,-73.943708,19587,Subscriber,1981.0,1,2017-07-01 00:00:03,0,0,36.0,1
2,328,2017-07-01 00:00:08,2017-07-01 00:05:37,3242,Schermerhorn St & Court St,40.691029,-73.991834,3397,Court St & Nelson St,40.676395,-73.998699,27937,Subscriber,1984.0,2,2017-07-01 00:00:08,0,0,33.0,1
3,2530,2017-07-01 00:00:11,2017-07-01 00:42:22,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,26066,Subscriber,1985.0,1,2017-07-01 00:00:11,0,0,32.0,1
4,2534,2017-07-01 00:00:15,2017-07-01 00:42:29,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,29408,Subscriber,1982.0,2,2017-07-01 00:00:15,0,0,35.0,1


In [18]:
# get rid of Nans so we can run our correlation tests
data = data.dropna(subset=['age'], how='all', axis=0)
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,hour,isDay,age,isBrooklyn
0,364,2017-07-01 00:00:00,2017-07-01 00:06:05,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,14744,Subscriber,1986.0,1,2017-07-01 00:00:00,0,0,31.0,0
1,2142,2017-07-01 00:00:03,2017-07-01 00:35:46,293,Lafayette St & E 8 St,40.730207,-73.991026,3425,2 Ave & E 104 St,40.789210,-73.943708,19587,Subscriber,1981.0,1,2017-07-01 00:00:03,0,0,36.0,1
2,328,2017-07-01 00:00:08,2017-07-01 00:05:37,3242,Schermerhorn St & Court St,40.691029,-73.991834,3397,Court St & Nelson St,40.676395,-73.998699,27937,Subscriber,1984.0,2,2017-07-01 00:00:08,0,0,33.0,1
3,2530,2017-07-01 00:00:11,2017-07-01 00:42:22,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,26066,Subscriber,1985.0,1,2017-07-01 00:00:11,0,0,32.0,1
4,2534,2017-07-01 00:00:15,2017-07-01 00:42:29,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,398,Atlantic Ave & Furman St,40.691652,-73.999979,29408,Subscriber,1982.0,2,2017-07-01 00:00:15,0,0,35.0,1


In [19]:
brooky = data.age[data['isBrooklyn'].values == 1]
manny = data.age[data['isBrooklyn'].values == 0]

In [20]:
size(manny)

282932

In [21]:
size(brooky)

2630495

In [22]:
Bsample = np.random.choice(brooky, size=282932, replace=False)

# 282932/200
# False so we don't redraw something we've used already
Bsample200 = np.random.choice(Bsample, size=1414, replace=False)
Msample200 = np.random.choice(manny, size=1414, replace=False)


# KS test

In [23]:
# for the entire dataset
scipy.stats.ks_2samp(brooky, manny)

# the closer to zero the more likely it is that the samples are drawn from the same distribution
# these are likely 
# null: test statistic is low, which means that they come from the same distribution, which means we reject the null (that they come from diff distributions)

Ks_2sampResult(statistic=0.097153717581210164, pvalue=0.0)

### What this means
Since the score is close to 0 and p value is low, they come from the same distribution. We can reject the null. 

# Pearson's test 

In [24]:
Bsample_sorted = np.sort(Bsample200)
Manny_sorted = np.sort(Msample200)

In [25]:
scipy.stats.pearsonr(Bsample_sorted, Manny_sorted)

(0.99272513922473482, 0.0)

### What it means
These are strongly correlated with p value of zero. We can reject the null. 

# Spearman's test

In [26]:
scipy.stats.spearmanr(Bsample_sorted, Manny_sorted)

SpearmanrResult(correlation=0.99885498467901879, pvalue=0.0)

### What it means
These are strongly correlated with p value of zero. We can reject the null. 